# Machine Learning in Python - Workshop 2

## 1. Setup

### 1.1 Packages
In the cell below we will load the core libraries we will be using for this workshop and setting some sensible defaults for our plot size and resolution.

In [2]:
# Display plots inline
%matplotlib inline

# Data libraries
import pandas as pd
import numpy as np

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting defaults
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['figure.dpi'] = 80

### 1.2 Data

To begin, we will examine a simple data set on the size and weight of a number of books. These data come from the `allbacks` data set from the `DAAG` package in R. Our goal is to model the weight of a book using some combination of the other features in the data. The included columns are as follows:

* `volume` - book volumes in cubic centimeters
* `area` - hard board cover areas in square centimeters
* `weight` - book weights in grams
* `cover` - a factor with levels `"hb"` hardback, `"pb"` paperback

We read the data into python using pandas,

In [3]:
books = pd.read_csv("daag_books.csv")
books

,volume,area,weight,cover
0,885,382,800,hb
1,1016,468,950,hb
2,1125,387,1050,hb
3,239,371,350,hb
4,701,371,750,hb
5,641,367,600,hb
6,1228,396,1075,hb
7,412,0,250,pb
8,953,0,700,pb
9,929,0,650,pb


---

### &diams; Exercise 1

Create a pairs plot of these data (make sure to include the `cover` column), describe any relationships you observe in the data.

---

## 1. Regression

We will begin by fitting a simple linear regression model for `weight` using exclusively `volume` as a feature. 

---

### &diams; Exercise 2

Create a scatter plot of these data describe any apparent relationship between these variables.

---

### 1.1 Least Squares

In lecture we discussed how we can represent a regression problem using matrix notation and we can derive a solution using least squares. We can express this as,

$$
\underset{\boldsymbol{\beta}}{\text{argmin}} \,\, \lVert \boldsymbol{y} - \boldsymbol{X} \boldsymbol{\beta} \rVert^2 
= \underset{\boldsymbol{\beta}}{\text{argmin}} \,\, (\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\beta})^\top(\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\beta})
$$

where,

$$
\underset{n \times 1}{\boldsymbol{y}} = \begin{pmatrix} y_1 \\ y_2 \\ \vdots \\ y_{n-1} \\ y_n \end{pmatrix} 
\qquad
\underset{n \times 2}{\boldsymbol{X}} = \begin{pmatrix} 1 & x_1 \\ 1 & x_2 \\ \vdots & \vdots \\ 1 & x_{n-1} \\ 1 & x_n \\ \end{pmatrix}
\qquad 
\underset{2 \times 1}{\boldsymbol{\beta}} = \begin{pmatrix} \beta_0 \\ \beta_1 \end{pmatrix}
$$

The solution to this optimization problem is,

$$
\boldsymbol{\beta} = \left(\boldsymbol{X}^\top\boldsymbol{X}\right)^{-1} \boldsymbol{X}^\top\boldsymbol{y}
$$

In Python we can construct the model matrix `X` by combining a column of ones, for the intercept, with our observed `volume` values. Similarly, `y` is a column vector of the `weight` values. In both cases we construct these objects as numpy array objects.

In [ ]:
y = np.array(books.weight)
print(y)

In [ ]:
X = np.c_[
    np.ones(len(y)),
    books.volume
]

print(X[:5])

Given the model matrix $(\boldsymbol{X})$ and observed outcomes $(\boldsymbol{y})$ we can then calculate the vector of solutions $(\boldsymbol{\beta})$ using numpy,

In [ ]:
from numpy.linalg import inv

beta = inv(X.T @ X) @ X.T @ y
print(beta)

Note that when using numpy `@` performs  matrix multiplication while `*` performs elementwise multiplication between arrays. Numpy matrix multiplication can also be written using `A.dot(B)` or `np.matmul(A,B)`.

We can calculate predictions from this model by calculating $\hat{y} = \boldsymbol{X} \boldsymbol{\beta}$. 

---

### &diams; Exercise 3

Calculate these predicted book weights and store them in the origin `books` data frame in a column called `weight_ls_pred`. Print out the updated version of the data frame with this new column added.

---

Given the predictions we can create a plot showing the models fit by overlaying a line plot of the predictions on top of the original scatter plot.

In [ ]:
sns.scatterplot(x="volume", y="weight", hue="cover", data=books)
sns.lineplot(x="volume", y="weight_ls_pred", color="black", data=books)

---

### 1.2 scikit-Learn

Constructing the model matrix by hand and calculating $\beta$ and model predictions using the least squares solution is less than ideal. As you might expect there are a number of higher level libraries that take care of many of these details. In this course we will be using the **scikit-learn** (**sklearn**) library to implement most of our machine learning models. As the semester progresses we will be learning about and implementing many different modeling methods. Additionally, we will also be learning how to use the larger data processing and workflow tools that are available in this library.

sklearn separates its various modeling tools into submodules organized by model type - for today we will be using the `LinearRegression` model from the `linear_model` submodule. Which we can import as follows,

In [ ]:
from sklearn.linear_model import LinearRegression

In general sklearn's models are implemented by first creating a model object, which is configured via constructor arguments, and then using that object to fit your data. As such, we will now create a linear regression model object `lr` and use it to fit our data. Once this object is created we use the `fit` method to obtain a model object fitted to our data. This model object then has various useful methods and attributes, including `intercept_` and `coef_` which contain our estimates for $\beta$.

In [ ]:
lr = LinearRegression()
l = lr.fit(
    X = np.array(books.volume).reshape(-1,1), # X must be a matrix so we need to reshape the column
    y = books.weight
)
b0 = l.intercept_
b1 = l.coef_[0]   # Subsetting in order to get a scalar value
beta = (b0, b1)

print(beta)

Using this default construction of `LinearRegression`, sklearn assumes that we have not included an intercept column (ones) in our model matrix and takes care of this for you. Additionally, since the intercept column is added the $\beta$ estimated for this particular column is stored separately, in the `intercept_` attribue.

I generally find this default behavior to be somewhat frustrating to work with, instead my preference is to handle all of the details of constructing the model matrix `X` myself and retrieving all `\beta` values (including the intercept) from `coef_` directly. For example, if we use the `X` and `y` variables we defined for the least squares example above and construct the `LinearRegression` object using `fit_intercept=False` then,

In [ ]:
l = LinearRegression(fit_intercept=False).fit(X = X, y = y)
beta = l.coef_

print(beta)

Note that this is the same answers we obtained above.

The model fit objects also provide additional useful methods for evaluating the model (`score`), more on this next week, and calculating predictions (`predict`). Using the later we can add another column of predictions to our data frame.

In [ ]:
books["weight_sk_pred"] = l.predict(X)
books

---

### &diams; Exercise 5

Do these results agree with the results we obtained when using the least squares method?

---

### 1.3 Residuals

One of the most useful tools for evaluating a model is to examine the residuals of that model. For any standard regression model the residual for observation $i$ is defined as $y_i - \hat{y}_i$ where $\hat{y}_i$ is the model's predicted value for observation $i$. In the case of linear regression
$\hat{y} = \boldsymbol{X} \boldsymbol{\beta}$.

---

### &diams; Exercise 6

Calculate the residual for each observation and store it in a column named `resid`. Using this new column create a residual plot (scatter plot of `volume` vs `resid`) for this model, be sure to color the points based on the `cover` type. 

---

### &diams; Exercise 7

Are there any particular issues we should be concerned about with this model based on what you see in the residual plot?

---

## 2. Regression with Categorical Variables



### 2.2 Dummy Coding

Based on these results, it should be clear that it is important that our model include information about whether or not a book is a hardback or paperback. As such, we need a way of encoding this information into our modeling framework. To do this we need a way of converting our string / categorical variable into a numeric representation that can be included in our model matrix.

The most common approach for doing this is called dummy coding, in the case of a binary categorical variable it involves picking one of the two levels of the categorical variable and encoding it as 1 and the other level as 0. With Python we can accomplish this by comparing our categorical vector to the value of our choice and then casting (converting) the result to an integer type.

For example if we wanted to code `hb` as 1 and `pb` as 0 we would do the following,

In [ ]:
books["cover_hb"] = (books.cover == "hb").astype(int) # Returns either 0 or 1
books

This is equivalent to using an indicator function in mathematical notation,

$$ 
\mathbb{1}_{hb_i} = 
\begin{cases}
1 & \text {if cover of book $i$ is hardback} \\
0 & \text {if cover of book $i$ is paperback}
\end{cases}
$$

Alternatively, we can defined the opposite of this where we code `hardback` as 0 and `paperback` as 1,


In [ ]:
books["cover_pb"] = (books.cover == "pb").astype(int) # Returns either 0 or 1
books

Now that we have recoded our categorical variable, `cover`, into a numerical variable we can fit a standard regression model with the form,

$$ y_i = \beta_0 + \beta_1 \, x_i + \beta_2 \, \mathbb{1}_{hb_i} $$

which we can represent in matrix form using, $\boldsymbol{y} = \boldsymbol{X} \boldsymbol{\beta}$
where $\boldsymbol{X} = \big[ \boldsymbol{1},\, \boldsymbol{x},\, \boldsymbol{\mathbb{1}_{hb}} \big]$.

Using Python, we can use the concatenate function with our 1s column, the `volume` column, and our new dummy coded indicator column, `cover_hb`,

In [ ]:
X = np.c_[np.ones(len(y)), books.volume, books.cover_hb]
l = LinearRegression(fit_intercept=False).fit(X, books.weight)

beta = l.coef_

print(beta)

This gives us a regression equation of the form,

$$ 
y_i = 13.9 + 0.72 \, x_i + 184.0 \, \mathbb{1}_{hb_i} 
$$

which can be rewritten as two separate line equations (one for each case of `cover`),

$$
y_i = \begin{cases}
        13.9 + 0.72 \, x_i & \text{if book cover $i$ is paperback} \\
        (13.9 + 184.0) + 0.72 \, x_i & \text{if book cover $i$ is hardback} \\
       \end{cases}.
$$

We can calculate prediction points along those lines using the following Python code in which we hard code the possible values of $\boldsymbol{\mathbb{1}_{hb_i}}$

In [ ]:
books["weight_hb_pred"] = l.predict(X)
books

and we can then plot both of these lines along with the observed data.

In [ ]:
sns.scatterplot(x="volume", y="weight", hue="cover", data=books)
sns.lineplot(x="volume", y="weight_hb_pred", hue="cover", data=books)

As well as create plot a residual plot of this new model,

In [ ]:
books["resid_hb"] = books.weight - books.weight_hb_pred
sns.scatterplot(x="volume", y="resid_hb", hue="cover", data=books)

---

### &diams; Exercise 8

Based on these regression fits, do you think the model including the dummy coded `cover` variable produces a "better" model than our first regression model which did not include `cover`? Explain. 

---

Note that by including a dummy variable in our model will change the interpretation of our regression coefficients. In this context,

* $\beta_0$ - This is the expected weight of a book with a `volume` of zero and a `hardback` indicator of zero, in other words a softcover book with zero volume.

* $\beta_1$ - This is the expected additional weight a book would have if its volume were to increase by 1 cm$^3$, all else being equal.

* $\beta_2$ - This is the expected additional weight a book would have if its hardcover indicator were to increase by 1, all else being equal. However, the hardcover indicator can only be 0 or 1 and hence this is the change in weight we would expect between a softcover book and a hardcover book with the same volume. In other words, hardcover books weight 184g more than softcover books.

Based on these interpretations we can see that the level that was coded as 0 (what is often called the reference level) gets folded into our intercept and the slope coefficient for the indicator provides the difference in intercept between the reference and the contrast level (level coded as 1).


---

### &diams; Exercise 9

Repeat the analysis above but this time fit a model using `pb` instead of `hb` in your model matrix. You should fit the new model as well as calculate the predictions for both paperback and hardback books.

---

### &diams; Exercise 10

What changed between the model with `cover_pb` vs the model with `cover_hb`? Specifically, comment on the values of $\beta_0$, $\beta_1$, and $\beta_2$ and their interpretations.


---

### 1.4 One hot encoding

A common approach for transforming categorical variables is know as one hot encoding, in which each level of the categorical variable is transformed into a new column with values of 0 or 1. This is equivalent to what we have done manually above.

Pandas has a built-in method for performing this on categorical columns. This is easiest to see with a simple example, below we construct a data frame `df` with a single column that we transform into a one hot encoded version using panda's `get_dummies` method.

In [ ]:
df = pd.DataFrame({"col": ["A","B","C","A", np.nan]})
df

In [ ]:
pd.get_dummies(df)

We can also perform typical statistical dummy coding by using the `drop_first=True` argument, which excludes the first column as a reference level.

In [ ]:
pd.get_dummies(df, drop_first=True)

Missing values can also be included as an additional category via the `dummy_na=True` argument.

In [ ]:
pd.get_dummies(df, dummy_na=True)

We can now use this approach with sklearn's linear regression model to simplify the process of creating our model with both `volume` and `cover`. Using `get_dummies` on a clean copy of the `books` data frame automatically replaces the `cover` column with `cover_hb` and `cover_pb`, which should match the columns we created by hand above. 

In [ ]:
books = pd.read_csv("daag_books.csv") # Reread in the data for a clean copy
books = pd.get_dummies(books)
books

---

### 1.3 Rank deficiency

Now lets consider the model where we naively include both `cover_hb` and `cover_pb` as well as an intercept column in our model matrix.

In [ ]:
X = np.c_[
    np.ones(len(y)), 
    books.volume, 
    books.cover_hb,
    books.cover_pb
]
l = LinearRegression(fit_intercept=False).fit(X, books.weight)


beta = l.coef_
print( beta )

---

### &diams; Exercise 11

Write out the equations that predict weight for hardback and paperback books according to this model.  


---

### &diams; Exercise 12

Are the solutions ($\beta$) given above unique? Can you find different values of $\beta_0$, $\beta_1$, $\beta_2$, and $\beta_3$ that would give you the same regression equations you wrote out in the previous exercise?

---

### &diams; Exercise 13

Solve for $\beta$ using the least squares solutions mentioned in Section 1. Do the solutions differ from sklearn's solutions? Do they make sense? Explain.

---